Perceptron Model for OCR and Face Detection

OCR on Digits

In [156]:
import numpy as np

# Read the file
file_path = 'digitdata/trainingimages'  # Replace with the path to your digits file
with open(file_path, 'rb') as file:
    lines = file.readlines()

# Initialize variables
image_data = []
images = []

# Parse the lines and populate the list of images
for i, line in enumerate(lines):
    line = line.decode()  # Convert bytes to string and remove newline character
    lineSplit= line.split("\n")
    line = "".join(lineSplit)
    
    # Check for non-empty line
    if line:
        row_data = [1 if c == '#' else 0 for c in line]  # Convert symbols to pixel values
        image_data.append(row_data)
        
        # Check for end of image (28 lines)
        if len(image_data) == 28:
            # Convert the list of lists (28x28) to a numpy array
            image_array = np.array(image_data)
            
            # Extract features by counting 0s in each 7x7 square
            features = []
            for i in range(0, 28, 7):
                for j in range(0, 28, 7):
                    square = image_array[i:i+7, j:j+7]
                    count_zeros = np.sum(square == 1)
                    features.append(count_zeros)
            
            # Flatten the 4x4 feature matrix
            features = np.array(features).flatten()
            
            images.append(features)
            image_data = []  # Reset image_data
    else:
        # Check for start of new image (empty line followed by line with symbols)
        if i < len(lines) - 1 and lines[i + 1].decode():  # Check if next line is not empty
            image_data = []  # Reset image_data

# Convert the list of images to a matrix
X = np.array(images)
print(X.shape)  # Should print (number_of_images, 16)


fileLab = 'digitdata/traininglabels'
with open(fileLab, 'rb') as file:
    lineLab = file.readlines()
labels = []
for i, line in enumerate(lineLab):
    line = line.decode()
    lineSplit= line.split("\n")
    line = "".join(lineSplit)
    labels.append(int(line))
Y = labels
print(len(Y))

(5000, 16)
5000


In [157]:
import numpy as np

# Read the file
file_path = 'digitdata/testimages'  # Replace with the path to your digits file
with open(file_path, 'rb') as file:
    lines = file.readlines()

# Initialize variables
image_data = []
images = []

# Parse the lines and populate the list of images
for i, line in enumerate(lines):
    line = line.decode()  # Convert bytes to string and remove newline character
    lineSplit= line.split("\n")
    line = "".join(lineSplit)
    
    # Check for non-empty line
    if line:
        row_data = [1 if c == '#' else 0 for c in line]  # Convert symbols to pixel values
        image_data.append(row_data)
        
        # Check for end of image (28 lines)
        if len(image_data) == 28:
            # Convert the list of lists (28x28) to a numpy array
            image_array = np.array(image_data)
            
            # Extract features by counting 0s in each 7x7 square
            features = []
            for i in range(0, 28, 7):
                for j in range(0, 28, 7):
                    square = image_array[i:i+7, j:j+7]
                    count_zeros = np.sum(square == 1)
                    features.append(count_zeros)
            
            # Flatten the 4x4 feature matrix
            features = np.array(features).flatten()
            
            images.append(features)
            image_data = []  # Reset image_data
    else:
        # Check for start of new image (empty line followed by line with symbols)
        if i < len(lines) - 1 and lines[i + 1].decode():  # Check if next line is not empty
            image_data = []  # Reset image_data

# Convert the list of images to a matrix
X_Test = np.array(images)
print(X_Test)
print(X_Test.shape)  # Should print (number_of_images, 16)

fileLab = 'digitdata/testlabels'
with open(fileLab, 'rb') as file:
    lineLab = file.readlines()
labels = []
for i, line in enumerate(lineLab):
    line = line.decode()
    lineSplit= line.split("\n")
    line = "".join(lineSplit)
    labels.append(int(line))
Y_Test = labels
print(len(Y_Test))

[[ 0  0  0 ... 10  0  0]
 [ 0  1 10 ...  7 12  0]
 [ 0  6 18 ... 13  4  0]
 ...
 [ 0  4  7 ... 10  9  0]
 [ 0  0  3 ...  0  7  0]
 [ 0  0  3 ...  7  9  0]]
(1000, 16)
1000


In [133]:
import numpy as np

class CustomPerceptron:
    def __init__(self, input_size, output_size):
        self.weights = np.zeros((output_size, input_size))
        self.bias = np.zeros(output_size)
    
    def train(self, X, y, epochs):
        for epoch in range(epochs):
            for i in range(X.shape[0]):
                # Forward pass
                #print("X[i]",len(X[i]))
                #print("self.weights",(self.weights.shape))
                """
                print("weight updates")
                print(self.weights.shape)
                print("weights",self.weights)
                print("X[i]",len(X[i]))
                #print(X[i].shape)
                """
                weighted_sum = np.dot(self.weights,X[i]) + self.bias
                #print("weighted_sums", weighted_sum)
                prediction = np.argmax(weighted_sum)
                #print("Prediction:",prediction)
                
                # Weight update based on feature values
                #print("y[i]",y[i])
                if y[i] != prediction:
                    
                    #print("Prediction weight before:")
                    #print(self.weights[prediction, :])
                    self.weights[prediction, :] -= X[i]
                    #print("Prediction weight after:")
                    #print(self.weights[prediction, :])
                    self.bias[prediction] -= 1
                    #print("Actual weight before:")
                    #print(self.weights[y[i],:])
                    self.weights[y[i], :] += X[i]
                    #print("Actual weight after:")
                    #print(self.weights[y[i], :])
                    self.bias[y[i]] += 1
    
    def predict(self, X):
        predictions = []
        for i in range(X.shape[0]):
            weighted_sum = np.dot(self.weights, X[i]) + self.bias
            #print(weighted_sum)
            prediction = (np.argmax(weighted_sum))
            predictions.append(prediction)
        return predictions

In [158]:
X_train = X[:int(X.shape[0]*1.0), :]
Y_train = Y[:int(len(Y)*1.0)]
X_test = X_Test[:int(X_Test.shape[0]*1.0), :]
Y_test = Y_Test[:int(len(Y_Test)*1.0)]
#print(X_train)
#print(Y_train)
#print(X_test)
#print(Y_test)


In [160]:
# Initialize and train Custom Perceptron for OCR
ocr_perceptron = CustomPerceptron(input_size=16, output_size=10)
ocr_perceptron.train(X_train, Y_train, 40)

# Predict and evaluate
ocr_predictions = ocr_perceptron.predict(X_test)
#print(ocr_predictions)
CorrectPredictionCount = 0
for i in range(len(ocr_predictions)):
    if ocr_predictions[i] == Y_test[i]:
        #print("Prediction:",ocr_predictions[i],"Actual:",Y_test[i])
        CorrectPredictionCount += 1
Ocr_Accuracy = CorrectPredictionCount/len(ocr_predictions)
#ocr_accuracy = np.mean(ocr_predictions == Y_test)
print(f'OCR Accuracy: {Ocr_Accuracy * 100:.2f}%')

OCR Accuracy: 56.80%


Face Detection

In [ ]:
import numpy as np

# Read the file
file_path = 'facedata/facedatatrain'  # Replace with the path to your digits file
with open(file_path, 'rb') as file:
    lines = file.readlines()

# Initialize variables
image_data = []
images = []

# Parse the lines and populate the list of images
for i, line in enumerate(lines):
    line = line.decode()  # Convert bytes to string and remove newline character
    lineSplit= line.split("\n")
    line = "".join(lineSplit)
    
    # Check for non-empty line
    if line:
        row_data = [1 if c == '#' else 0 for c in line]  # Convert symbols to pixel values
        image_data.append(row_data)
        
        # Check for end of image (28 lines)
        if len(image_data) == 28:
            # Convert the list of lists (28x28) to a numpy array
            image_array = np.array(image_data)
            
            # Extract features by counting 0s in each 7x7 square
            features = []
            for i in range(0, 28, 7):
                for j in range(0, 28, 7):
                    square = image_array[i:i+7, j:j+7]
                    count_zeros = np.sum(square == 1)
                    features.append(count_zeros)
            
            # Flatten the 4x4 feature matrix
            features = np.array(features).flatten()
            
            images.append(features)
            image_data = []  # Reset image_data
    else:
        # Check for start of new image (empty line followed by line with symbols)
        if i < len(lines) - 1 and lines[i + 1].decode():  # Check if next line is not empty
            image_data = []  # Reset image_data

# Convert the list of images to a matrix
X = np.array(images)
print(X.shape)  # Should print (number_of_images, 16)


fileLab = 'digitdata/traininglabels'
with open(fileLab, 'rb') as file:
    lineLab = file.readlines()
labels = []
for i, line in enumerate(lineLab):
    line = line.decode()
    lineSplit= line.split("\n")
    line = "".join(lineSplit)
    labels.append(int(line))
Y = labels
print(len(Y))

In [ ]:
# Face Detection Example
# Assuming you have face detection data (features and labels)
# X_face, y_face = load_face_data()  # Load your face detection data

# Initialize and train Custom Perceptron for Face Detection
face_perceptron = CustomPerceptron(input_size=X_face.shape[1], output_size=2)
face_perceptron.train(X_face, y_face)

# Predict and evaluate
face_predictions = face_perceptron.predict(X_face)
face_accuracy = np.mean(face_predictions == y_face)
print(f'Face Detection Accuracy: {face_accuracy * 100:.2f}%')